 # Delta Lake tables 
 Use this notebook to explore Delta Lake functionality

In [1]:
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

# define the schema
schema = StructType() \
.add("ProductID", IntegerType(), True) \
.add("ProductName", StringType(), True) \
.add("Category", StringType(), True) \
.add("ListPrice", DoubleType(), True)

df = spark.read.format("csv").option("header","true").schema(schema).load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 456bcd83-a3ed-4625-8a2b-87c7866f2a53)

## Create Delta tables

managed

In [2]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 4, Finished, Available, Finished)

external

In [6]:
df.write.format("delta").saveAsTable("external_products", path="abfss://delta_workspace@onelake.dfs.fabric.microsoft.com/practice.Lakehouse/Files/external_products")

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 8, Finished, Available, Finished)

managed vs external

In [7]:
%%sql
DESCRIBE FORMATTED managed_products;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 9, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [8]:
%%sql
DESCRIBE FORMATTED external_products;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 10, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [9]:
%%sql
DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Use SQL to create a Delta table

SQL magic command you can use SQL syntax instead of Pyspark. Here you will create a temporary view.

In [10]:
%%sql
CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
SELECT * FROM products;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 14, Finished, Available, Finished)

<Spark SQL result set with 295 rows and 4 fields>

## Explore table versioning

10% reduction in the price for mountain bikes

In [12]:
%%sql
UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

the history of transactions

In [13]:
%%sql
DESCRIBE HISTORY products;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 16, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

one containing the data after the price reduction, and the other showing the original version of the data

In [14]:
delta_table_path = 'Files/external_products'
# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a646a570-396a-40be-b8c6-b7a41c0ab1d4)

SynapseWidget(Synapse.DataFrame, ec1561cc-4cfc-4b16-8e84-a5b80d461f40)

## Analyze Delta table data with SQL queries

In [15]:
%%sql
-- Create a temporary view
CREATE OR REPLACE TEMPORARY VIEW products_view
AS
    SELECT Category, COUNT(*) AS NumProducts, MIN(ListPrice) AS MinPrice, MAX(ListPrice) AS MaxPrice, AVG(ListPrice) AS AvgPrice
    FROM products
    GROUP BY Category;

SELECT *
FROM products_view
ORDER BY Category;    

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 19, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 37 rows and 5 fields>

top 10 categories by number of products

In [16]:
%%sql
SELECT Category, NumProducts
FROM products_view
ORDER BY NumProducts DESC
LIMIT 10;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 20, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [17]:
from pyspark.sql.functions import col, desc

df_products = spark.sql("SELECT Category, MinPrice, MaxPrice, AvgPrice FROM products_view").orderBy(col("AvgPrice").desc())
display(df_products.limit(6))

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 054e8c95-c09b-4a78-ab27-e1581bf05ed6)

## Use Delta tables for streaming data

In [21]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data, True)

print("Source stream created...")

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 26, Finished, Available, Finished)

Source stream created...


In [23]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 28, Finished, Available, Finished)

Streaming to delta sink...


In [24]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 29, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

writes more hypothetical device data to the streaming source

In [25]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 30, Finished, Available, Finished)

True

In [26]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 31, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [27]:
deltastream.stop()

StatementMeta(, 66b84f82-15fb-4e7d-b38d-53e7be382959, 32, Finished, Available, Finished)

# Clean up resources
In this exercise, you’ve learned how to work with Delta tables in Microsoft Fabric.

If you’ve finished exploring your lakehouse, you can delete the workspace you created for this exercise.

In the bar on the left, select the icon for your workspace to view all of the items it contains.
In the … menu on the toolbar, select Workspace settings.
In the General section, select Remove this workspace.